## Preparation

imports and variables

In [6]:
import json
import os
import pandas as pd
import logging
import argparse
import csv
import configparser
from TM1py.Services import TM1Service
from TM1py.Objects import Dimension, Hierarchy, Element,Cube,ElementAttribute
from collections import defaultdict
from TM1py.Utils import CaseAndSpaceInsensitiveTuplesDict
import datetime

In [7]:
def get_tm1_service(instance_name):
    config = configparser.ConfigParser()
    config.read(r'config.ini')
    return TM1Service(**config[instance_name])

In [2]:
TM1_PARAMS = {
    "base_url": "https://localhost:15822",
    "user": "admin",
    "password": ""
}

In [3]:
TM1_CLOUD_PARAMS = {
    "base_url": "https://cubewise.planning-analytics.ibmcloud.com/tm1/api/plansamp",
    "user": "cubewise01_tm1_automation",
    "namespace": "LDAP",
    "password": "XI2b5FryLHR679",
    "ssl": True,
    "verify": True,
    "async_requests_mode": True
}

## Part 1

### load data from JSON and create a dimension



Please open the `products.json` file in `C:/resources/` with notepad. 

This document describes a dimension using JSON.



JSON is a popular format to exchange information between systems. 
It is commonly used in APIs as a format for data and metadata.

Without TM1py it would be hard to consume for TM1


Now let's read the JSON file with Python.

open `products.json` file with the `open` function and read it using the `json.load` function

```
with open("file-name", "r") as file:
    data = json.load(file)
```

Now use the `pd.json_normalize` function to turn it into a data frame

In [11]:
tm1 = get_tm1_service('Hello_World')
print(tm1.server.get_server_name())

Hello_World


Now connect to TM1 using 

```
tm1 = TM1Service(**TM1_PARAMS)
```

To test the connection call below function

```
print(tm1.server.get_server_name())
```

Once connected successfully, use the `update_or_create_hierarchy_from_dataframe` function in the `tm1.hierarchies` module to update the `Product` dimension in TM1 based on the data frame

```
tm1.hierarchies.update_or_create_hierarchy_from_dataframe(
        dimension_name="dimension name",
        hierarchy_name="hierarchy name",
        df=df)
```

In [12]:
with open("products.json", "r") as file:
    data = json.load(file)

In [13]:
data

[{'ProductID': 'P001',
  'ProductName': 'NextGen Tablets 1',
  'Category': {'MainCategory': 'Electronics',
   'SubCategory': 'Mobile Devices',
   'ProductType': 'Tablets'},
  'Brand': 'NextGen',
  'Price': 851.45,
  'Specifications': {'Color': 'Gold',
   'Storage': '256GB',
   'ScreenSize': '11.6 inches',
   'BatteryCapacity': '6000mAh'},
  'Features': {'Camera': {'Resolution': '20MP', 'Zoom': 'Optical 3x'},
   'Connectivity': {'Type': '5G', 'WiFi': 'WiFi 4'}},
  'ReleaseDate': '2023-07-02'},
 {'ProductID': 'P002',
  'ProductName': 'UltraTech Smartphones 2',
  'Category': {'MainCategory': 'Electronics',
   'SubCategory': 'Mobile Devices',
   'ProductType': 'Smartphones'},
  'Brand': 'UltraTech',
  'Price': 878.48,
  'Specifications': {'Color': 'Blue',
   'Storage': '384GB',
   'ScreenSize': '5.6 inches',
   'BatteryCapacity': '5000mAh'},
  'Features': {'Camera': {'Resolution': '8MP', 'Zoom': 'Optical 6x'},
   'Connectivity': {'Type': '5G', 'WiFi': 'WiFi 5'}},
  'ReleaseDate': '2023-01-

In [14]:
df = pd.DataFrame(data=data)

In [15]:
df

,ProductID,ProductName,Category,Brand,Price,Specifications,Features,ReleaseDate
0,P001,NextGen Tablets 1,"{'MainCategory': 'Electronics', 'SubCategory':...",NextGen,851.45,"{'Color': 'Gold', 'Storage': '256GB', 'ScreenS...","{'Camera': {'Resolution': '20MP', 'Zoom': 'Opt...",2023-07-02
1,P002,UltraTech Smartphones 2,"{'MainCategory': 'Electronics', 'SubCategory':...",UltraTech,878.48,"{'Color': 'Blue', 'Storage': '384GB', 'ScreenS...","{'Camera': {'Resolution': '8MP', 'Zoom': 'Opti...",2023-01-24
2,P003,SmartOne Smartwatches 3,"{'MainCategory': 'Electronics', 'SubCategory':...",SmartOne,525.38,"{'Color': 'Blue', 'Storage': '16GB', 'ScreenSi...","{'Camera': {'Resolution': '12MP', 'Zoom': 'Opt...",2023-03-21
3,P004,NextGen Smartwatches 4,"{'MainCategory': 'Electronics', 'SubCategory':...",NextGen,293.36,"{'Color': 'Red', 'Storage': '16GB', 'ScreenSiz...","{'Camera': {'Resolution': '8MP', 'Zoom': 'Opti...",2023-11-03
4,P005,NextGen Smartwatches 5,"{'MainCategory': 'Electronics', 'SubCategory':...",NextGen,474.38,"{'Color': 'White', 'Storage': '16GB', 'ScreenS...","{'Camera': {'Resolution': '12MP', 'Zoom': 'Opt...",2023-05-07
...,...,...,...,...,...,...,...,...
95,P096,SmartOne Smartphones 96,"{'MainCategory': 'Electronics', 'SubCategory':...",SmartOne,900.31,"{'Color': 'Gold', 'Storage': '192GB', 'ScreenS...","{'Camera': {'Resolution': '8MP', 'Zoom': 'Opti...",2023-04-14
96,P097,TechCorp Smartphones 97,"{'MainCategory': 'Electronics', 'SubCategory':...",TechCorp,560.22,"{'Color': 'Black', 'Storage': '384GB', 'Screen...","{'Camera': {'Resolution': '12MP', 'Zoom': 'Opt...",2023-09-13
97,P098,NextGen Smartwatches 98,"{'MainCategory': 'Electronics', 'SubCategory':...",NextGen,911.61,"{'Color': 'Blue', 'Storage': '16GB', 'ScreenSi...","{'Camera': {'Resolution': '16MP', 'Zoom': 'Opt...",2023-04-20
98,P099,NextGen Smartphones 99,"{'MainCategory': 'Electronics', 'SubCategory':...",NextGen,676.95,"{'Color': 'Silver', 'Storage': '64GB', 'Screen...","{'Camera': {'Resolution': '12MP', 'Zoom': 'Opt...",2023-10-27


Please review the result in TM1

The dimension should be complete, but there are no hierarchies or consolidations in it yet!

We can manipulate the data frame and add special level columns to it. Then TM1py will create the hierarchy accordingly.

To make sure each product has `Level000` as a root parent you can write this:

```
df["Level000"] = "All Products"
```

We can use the existing columns `"Category.ProductType"`, `"Category.SubCategory"` , `"Category.MainCategory"` in the data frame to define a good hierarchy. 

Please create 3 new columns in the dataframe based on the existing 3 columns mentioned above, like this:

```
df["Level001"] = df["existing column"]
df["Level002"] = df["existing column"]
df["Level003"] = df["existing column"]
```

Make sure the order is sensible. You can always print out your dataframe to view it as a table

Now use the `update_or_create_hierarchy_from_dataframe` function in the `tm1.hierarchies` module to update the `Product` dimension in TM1 again

```
tm1.hierarchies.update_or_create_hierarchy_from_dataframe(
        dimension_name="dimension name",
        hierarchy_name="hierarchy name",
        df=df,
        unwind=True)
```

You can pass `unwind=True` to unwind/break all existing consolidation in the dimension

-----

## Part 2

### load data from JSON and write to cube



open `sales.json` file and read it using the `json` module

```
with open("file-name", "r") as file:
    data = json.load(file)
```

Now use the `pd.json_normalize` function to turn it into a data frame

Once it is a dataframe you can call `df.shape()` or `df.sample(5)` to get an overview of the data

Before we can write the dataframe to our Sales cube in TM1, we need to make sure the data has the right granularity and the columns are in the right order.

Please add a column to the dataframe for the Version and assign all records to "Actual"
You can use this syntax

```
df["Column"] = "Value"
```


Please also order the columns according to the dimension in the cube

```
df = df[["Month", "Version", "Region", ..., "Value"]]
```

The data frame should looks like this

|        | Month   | Version   | Region      | Customer          | ProductID   | SalesChannel   | Measure        |   Value |
|-------:|:--------|:----------|:------------|:------------------|:------------|:---------------|:---------------|--------:|
|  24811 | 2022-03 | Actual    | Italy       | Jackson Corp      | P005        | Wholesale      | Sales Quantity |     486 |
|  91695 | 2023-03 | Actual    | Russia      | Garcia Devices    | P067        | Retail         | Sales Quantity |     456 |
|   1106 | 2023-12 | Actual    | New Zealand | Williams Tech     | P060        | Retail         | Sales Quantity |     172 |


Now you can use the `write_dataframe` function in the `tm1.cells` module to write to the Sales cube with below syntax

```
tm1.cells.write_dataframe(cube_name="cube name", data=df, use_blob=True)
```

Make sure to pass `use_blob=True` as this speeds up the operation

-----

## Part 3: The Challenge

### Synchronize local TM1 with cloud TM1 instance


In the top of this notebook you find a variable called `TM1_CLOUD_PARAMS`.
Please first verify that you can connect to this instance with TM1py.

In Part 3 you need to write the 2023 actuals from the local Sales cube to a budget version in an aggregated cube in the cloud for 2024 with a 10% increase on each value.



**Step 1**: extract 2023 actual data from Sales cube to a data frame, using MDX

hint: use Arc to generate the MDX

hint: use the `tm1.cells.execute_mdx_dataframe` function to run the MDX and get a dataframe

**Step 2**: increase the values by 10% 

hint: `df["column name"] = df["column name"] * 1.1`


**Step3**: change the value in the version column from `"actual"` to `"Budget your-name"`

**Step4**: change all periods from 2023 to 2024

hint: `df["column name"] = df["column name"].str.replace("old value", "new value")`

**Step5**: Before you write, make sure to crate the new budget element in the version dimension. 

hint:

```
    from TM1py import Element
    new_version_element = "Budget Marius"

    if tm1_cloud.elements.exists(dimension_name="Version", hierarchy_name="Version", element_name=new_version_element):
        tm1_cloud.elements.create("Version", "Version", element=Element(new_version_element, "Numeric"))
```


**Step6**: write the dataframe to the Sales cube in the TM1 cloud instance

make sure the column order is in line with the cube dimension order

-----

## Part 4: Optional

- Create product attributes with their correct type (e.g., `Product Name` shoud be Alias). _Hint: checkout the docs on the `update_or_create_hierarchy_from_dataframe` function_

- Clear a slice in the `Sales` cube before the write operation. _hint: use bedrock or tm1py's clear_with_mdx function_

- In the cloud instance you find another cube: `Sales Summary`. This cube has all the same dimensions as `Sales` except for `Customer`. Please read a new slice from your source cube, using MDX, with the aggregated values for `All Customers` and write the data to `Sales Summary` in the cloud instance

- Make sure the `product` dimension is sync between the two instances! Use the `get_elements_dataframe` function in `tm1.elements` to retrieve the dimension as a dataframe from the source instance. Use the `update_or_create_hierarchy_from_dataframe` function in `tm1.hierarchies` to update the Product dimension in the target TM1 instance
